In [1]:
import pandas as pd
import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import pickle

In [2]:
# Parameters
FILE_PATH = './Dataset/EURUSD/EURUSD_M30_features+label_v.2.1.csv'
COLUMNS = ['Close', 'SMA200', 'SMA13', 'RSI14']
LABEL = 'signal'
SEQ_LENGTH = 20
NUM_LAYERS = 4
NUM_HEADS = 4
BATCH_SIZE = 1024
EPOCHS = 50
DROPOUT = 0.4
LEARNING_RATE = 0.01
RANDOM_STATE = 13
TRAIN_SHUFFLE = True
TEST_SHUFFLE = True
TRAIN_TEST_RATIO = 0.2
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
# Load and preprocess data
data = pd.read_csv(FILE_PATH)
data = data[COLUMNS + [LABEL]]
data[COLUMNS] = data[COLUMNS].round(4)

In [4]:
# Normalize features
scaler = MinMaxScaler()
data[COLUMNS] = scaler.fit_transform(data[COLUMNS])

In [5]:
# Adjust labels
data[LABEL] = data[LABEL] - 1

In [6]:
# Create sequences
class ForexDataset(Dataset):
    def __init__(self, data, seq_length):
        self.features = []
        self.labels = []
        for i in range(len(data) - seq_length):
            self.features.append(data.iloc[i:i + seq_length][COLUMNS].values)
            self.labels.append(data.iloc[i + seq_length][LABEL])
        self.features = torch.tensor(self.features, dtype=torch.float32)
        self.labels = torch.tensor(self.labels, dtype=torch.long)

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        return self.features[idx], self.labels[idx]


In [7]:
# Split data
train_data, test_data = train_test_split(data, test_size=TRAIN_TEST_RATIO, random_state=RANDOM_STATE)

In [8]:
train_dataset = ForexDataset(train_data, SEQ_LENGTH)
test_dataset = ForexDataset(test_data, SEQ_LENGTH)

C:\Users\Stephen\AppData\Local\Temp\ipykernel_4840\2456821998.py:9: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\utils\tensor_new.cpp:281.)
  self.features = torch.tensor(self.features, dtype=torch.float32)


In [9]:
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=TRAIN_SHUFFLE)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=TEST_SHUFFLE)

In [10]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=5000):
        super(PositionalEncoding, self).__init__()
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-torch.log(torch.tensor(10000.0)) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:, :x.size(1), :]
        return x

In [11]:
class EnhancedTransformerModel(nn.Module):
    def __init__(self, input_dim, num_classes, seq_length, num_layers, num_heads, dropout):
        super(EnhancedTransformerModel, self).__init__()
        self.positional_encoding = PositionalEncoding(input_dim)
        self.norm1 = nn.LayerNorm(input_dim)
        self.encoder_layer = nn.TransformerEncoderLayer(
            d_model=input_dim, nhead=num_heads, dropout=dropout, batch_first=True
        )
        self.transformer = nn.TransformerEncoder(self.encoder_layer, num_layers=num_layers)
        self.fc1 = nn.Linear(seq_length * input_dim, 128)
        self.fc2 = nn.Linear(128, num_classes)
        self.dropout = nn.Dropout(dropout)
        self.activation = nn.ReLU()

    def forward(self, x):
        x = self.positional_encoding(x)
        x = self.norm1(x)
        x = self.transformer(x)
        x = x.flatten(start_dim=1)
        x = self.dropout(self.activation(self.fc1(x)))
        return self.fc2(x)

In [12]:
model = EnhancedTransformerModel(input_dim=len(COLUMNS), num_classes=3, seq_length=SEQ_LENGTH, num_layers=NUM_LAYERS, num_heads=NUM_HEADS, dropout=DROPOUT).to(DEVICE)

In [13]:
# Loss and optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=LEARNING_RATE, steps_per_epoch=len(train_loader), epochs=EPOCHS)
class_counts = data[LABEL].value_counts().to_dict()
total_samples = sum(class_counts.values())
class_weights = {cls: total_samples / count for cls, count in class_counts.items()}
weights = torch.tensor([class_weights[i] for i in range(3)], dtype=torch.float32).to(DEVICE)
criterion = nn.CrossEntropyLoss(weight=weights)

In [ ]:
for epoch in range(EPOCHS):
    model.train()
    train_loss, train_preds, train_labels = 0, [], []
    for features, labels in train_loader:
        features, labels = features.to(DEVICE), labels.to(DEVICE)
        optimizer.zero_grad()
        outputs = model(features)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        scheduler.step()  # Update learning rate
        train_loss += loss.item()
        train_preds.extend(torch.argmax(outputs, dim=1).cpu().numpy())
        train_labels.extend(labels.cpu().numpy())

    train_acc = accuracy_score(train_labels, train_preds)
    train_f1 = f1_score(train_labels, train_preds, average='weighted')

    model.eval()
    test_preds, test_labels = [], []
    with torch.no_grad():
        for features, labels in test_loader:
            features, labels = features.to(DEVICE), labels.to(DEVICE)
            outputs = model(features)
            test_preds.extend(torch.argmax(outputs, dim=1).cpu().numpy())
            test_labels.extend(labels.cpu().numpy())

    test_acc = accuracy_score(test_labels, test_preds)
    test_f1 = f1_score(test_labels, test_preds, average='weighted')

    print(f"Epoch {epoch + 1}/{EPOCHS}, "
          f"Train Loss: {train_loss:.4f}, "
          f"Train Acc: {train_acc:.4f}, Train F1: {train_f1:.4f}, "
          f"Test Acc: {test_acc:.4f}, Test F1: {test_f1:.4f}")


Epoch 1/50, Train Loss: 173.0687, Train Acc: 0.3413, Train F1: 0.3510, Test Acc: 0.2550, Test F1: 0.1036
Epoch 2/50, Train Loss: 172.5692, Train Acc: 0.3639, Train F1: 0.3688, Test Acc: 0.4997, Test F1: 0.3330
Epoch 3/50, Train Loss: 172.5570, Train Acc: 0.3748, Train F1: 0.3750, Test Acc: 0.4997, Test F1: 0.3330
Epoch 4/50, Train Loss: 172.5208, Train Acc: 0.3788, Train F1: 0.3757, Test Acc: 0.2453, Test F1: 0.0966
Epoch 5/50, Train Loss: 172.5122, Train Acc: 0.3871, Train F1: 0.3801, Test Acc: 0.2550, Test F1: 0.1036
Epoch 6/50, Train Loss: 172.4940, Train Acc: 0.3241, Train F1: 0.3325, Test Acc: 0.2550, Test F1: 0.1036
Epoch 7/50, Train Loss: 172.5104, Train Acc: 0.3087, Train F1: 0.3116, Test Acc: 0.2453, Test F1: 0.0966
Epoch 8/50, Train Loss: 172.4983, Train Acc: 0.3814, Train F1: 0.3777, Test Acc: 0.2550, Test F1: 0.1036
Epoch 9/50, Train Loss: 172.4884, Train Acc: 0.3298, Train F1: 0.3340, Test Acc: 0.2550, Test F1: 0.1036
Epoch 10/50, Train Loss: 172.4969, Train Acc: 0.3546, T